# Enorm Dienstplan automatisiert erstellen

## 0. Import libraries and data

In [4]:
import numpy as np
import pandas as pd


In [12]:
mitarbeitende_df = pd.read_csv('data/Input_Daten_Dienstplan - Mitarbeitende.csv')
wochenstunden_df = pd.read_csv('data/Input_Daten_Dienstplan - Wochenstunden.csv')
wochenstunden_df.head()
wochenstunden_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Mitarbeitenden_ID  19 non-null     int64
 1   Wochenstunden      19 non-null     int64
dtypes: int64(2)
memory usage: 432.0 bytes


## 1. Create simple Dienstplan

Create a dataframe with 8 hour slots ('Schichten') in a 24 hour day for a 30 days month where from the dataframe wochenstunden_df each employee ('Mitarbeitenden_ID') is randomly spread into the 'Schichten' and they have at least 11 hours breaks between their 'Schichten' and they are close to working their number of working hours per week ('Wochenstunden') found in wochenstunden_df. 

In [35]:
import pandas as pd
import numpy as np

# Load employee and working hours data
mitarbeitende_df = pd.read_csv('data/Input_Daten_Dienstplan - Mitarbeitende.csv')
wochenstunden_df = pd.read_csv('data/Input_Daten_Dienstplan - Wochenstunden.csv')

# Define parameters
num_days = 30
slot_types = ['morning', 'midday', 'evening', 'night']
hours_of_slot = 8

# Generate dates
dates = pd.date_range(start='2024-04-01', periods=num_days)
dates_repeated = [date for date in dates for _ in slot_types]

# Create Dienstplan DataFrame
Dienstplan_df = pd.DataFrame({'date': dates_repeated, 'slot_type': slot_types * num_days})

# Add additional column 'employee_id'
Dienstplan_df['employee_id'] = ''

# Randomly assign employees to slots
for index, row in Dienstplan_df.iterrows():
    # Select random employee ID
    random_employee_id = np.random.choice(mitarbeitende_df['Mitarbeitenden_ID'])
    Dienstplan_df.at[index, 'employee_id'] = random_employee_id

# Display the head of Dienstplan_df
print(Dienstplan_df.head())


        date slot_type employee_id
0 2024-04-01   morning         117
1 2024-04-01    midday         102
2 2024-04-01   evening         104
3 2024-04-01     night         116
4 2024-04-02   morning         109
